In [8]:
!pip install openpyxl --upgrade
!pip install openpyxl_xlsx --upgrade
!pip install pyexcel --upgrade

ERROR: Could not find a version that satisfies the requirement openpyxl_xlsx (from versions: none)
ERROR: No matching distribution found for openpyxl_xlsx


In [12]:
excel_file = "../SampleData/epl.xlsx"

In [13]:
import tempfile
from zipfile import ZipFile
import shutil
import os
from fnmatch import fnmatch


def change_in_zip(file_name, name_filter, change):
    tempdir = tempfile.mkdtemp()
    try:
        tempname = os.path.join(tempdir, "new.zip")
        with ZipFile(file_name, "r") as r, ZipFile(tempname, "w") as w:
            for item in r.infolist():
                data = r.read(item.filename)
                if fnmatch(item.filename, name_filter):
                    data = change(data)
                w.writestr(item, data)
        shutil.move(tempname, file_name)
    finally:
        shutil.rmtree(tempdir)


change_in_zip(
    excel_file,
    name_filter="xl/worksheets/*.xml",  # the problematic property is found in the worksheet xml files
    change=lambda d: d.replace(b' synchVertical="1"', b" "),
)

In [14]:
import pandas as pd
import numpy as np

In [15]:
epl = pd.read_excel(excel_file)
epl

,season_id,match_id,home_team_id,home_team_name,away_team_id,away_team_name,id,event_id,date,time,...,sec,team_id,player_id,playerName,playerPosition,x,y,type,description,outcome
0,2012,441999,3,Arsenal,56,Sunderland,688830980,1,18Aug2012,14:31:54,...,0,3,NaN,NaN,NaN,0.0,0.0,34,Team set up,1
1,2012,441999,3,Arsenal,56,Sunderland,1399442519,1,18Aug2012,14:33:35,...,0,56,NaN,NaN,NaN,0.0,0.0,34,Team set up,1
2,2012,441999,3,Arsenal,56,Sunderland,1641098842,2,18Aug2012,15:01:59,...,0,3,NaN,NaN,NaN,0.0,0.0,32,Start,1
3,2012,441999,3,Arsenal,56,Sunderland,1755825990,2,18Aug2012,15:01:59,...,0,56,NaN,NaN,NaN,0.0,0.0,32,Start,1
4,2012,441999,3,Arsenal,56,Sunderland,164697335,3,18Aug2012,15:02:00,...,1,3,17733.0,"Podolski, Lukas",Striker,50.0,50.0,1,Pass,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648878,2012,442378,111,Wigan Athletic,7,Aston Villa,428661649,905,19May2013,17:51:58,...,35,111,NaN,NaN,NaN,0.0,0.0,30,End,1
648879,2012,442378,111,Wigan Athletic,7,Aston Villa,1411196656,907,19May2013,18:00:29,...,0,111,NaN,NaN,NaN,0.0,0.0,30,End,1
648880,2012,442378,111,Wigan Athletic,7,Aston Villa,948803408,908,19May2013,18:00:29,...,0,111,NaN,NaN,NaN,0.0,0.0,37,Collection End,1
648881,2012,442378,111,Wigan Athletic,7,Aston Villa,803034702,810,19May2013,18:00:30,...,0,7,NaN,NaN,NaN,0.0,0.0,30,End,1
